# Testing  weights learned in [Inceptionv3_21](Inceptionv3_21.ipynb)

In [1]:
import os
import pickle

import numpy as np

from keras import backend as K
from keras.layers import Conv2D, Dropout, Dense
from keras.models import Sequential, model_from_json
from keras.preprocessing.image import load_img, img_to_array

from cars_utils import time_save_model, plot_metrics, pickle_variable_to_path

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Image, directory, and batch setup

In [2]:
# Image dimensions:
IMG_WIDTH, IMG_HEIGHT = 299, 299

WORKING_DIR = os.getcwd()
BASE = os.path.join(WORKING_DIR, 'data', 'cars_dataset', 'keras')
SAVE_DIR = os.path.join(WORKING_DIR, 'data', 'InceptionV3')
PREDICT_DIR = os.path.join(WORKING_DIR, 'data', 'predict_me')
LOOKUP_PATH = os.path.join(SAVE_DIR, 'I_15_lookup_dict.pkl')

NB_CLASSES = 196

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

## Model definition

In [6]:
from keras.applications import InceptionV3

conv_base = InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=INPUT_SHAPE
)

In [7]:
pred_layer_config = {
    'activation': 'softmax',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
                           'config': {
                               'distribution': 'uniform',
                               'mode': 'fan_avg',
                               'scale': 1.0,
                               'seed': 8}
                          },
    'kernel_regularizer': None,
    'name': 'predictions',
    'trainable': True,
    'units': NB_CLASSES,
    'use_bias': True}

model = Sequential()
model.add(conv_base)
model.add(Dropout(0.5, seed=21))
model.add(Dense(**pred_layer_config))

## First round of training

In [8]:
WEIGHT_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_0_10.h5')
model.load_weights(WEIGHT_PATH)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2048)              21802784  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
predictions (Dense)          (None, 196)               401604    
Total params: 22,204,388
Trainable params: 22,169,956
Non-trainable params: 34,432
_________________________________________________________________


### Predict!

In [13]:
test_file_list = [f for f in os.listdir(PREDICT_DIR) if\
                   os.path.isfile(os.path.join(PREDICT_DIR, f))]

with open(LOOKUP_PATH, 'rb') as f:
    lookup_dict = pickle.load(f)

for test_file in test_file_list:
    file_path = os.path.join(PREDICT_DIR, test_file)
    input_image = load_img(file_path, 
                           target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_image = img_to_array(input_image)
    # the images are scaled during training so they need to be scaled for predictions too
    input_image = input_image/255.
    input_image = np.expand_dims(input_image, axis=0)
    prediction = model.predict(input_image)
    top_3 = prediction[0].argsort()[-3:][::-1]
    print('\nPredictions for {}:'.format(test_file))
    for idx in top_3:
        print(lookup_dict.get(idx))


Predictions for Audi-100_Sedan-1994.jpg:
Mercedes_Benz-300_Class_Convertible-1993
Lincoln-Town_Car_Sedan-2011
Audi-100_Sedan-1994

Predictions for Aston_Martin-Martin_V8_Vantage_Convertible-2012.jpg:
Aston_Martin-Martin_Virage_Convertible-2012
BMW-6_Series_Convertible-2007
Chevrolet-Corvette_ZR1-2012

Predictions for Dodge-Ram_Pickup_3500_Crew_Cab-2010.jpg:
Dodge-Ram_Pickup_3500_Crew_Cab-2010
Ford-F_450_Super_Duty_Crew_Cab-2012
Ford-Expedition_EL_SUV-2009

Predictions for AM-General_Hummer_SUV-2000.jpg:
AM-General_Hummer_SUV-2000
HUMMER-H2_SUT_Crew_Cab-2009
Ford-F_450_Super_Duty_Crew_Cab-2012


## ROUND 2 of training (fine-tuned once)

In [3]:
JSON_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT1_10_20.json')
with open (JSON_PATH, 'r') as json_file:
    loaded_model_json = json_file.read()

json_model = model_from_json(loaded_model_json)

In [4]:
WEIGHT_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT1_10_20.h5')
json_model.load_weights(WEIGHT_PATH)

### Predict!

In [5]:
test_file_list = [f for f in os.listdir(PREDICT_DIR) if\
                   os.path.isfile(os.path.join(PREDICT_DIR, f))]

with open(LOOKUP_PATH, 'rb') as f:
    lookup_dict = pickle.load(f)

for test_file in test_file_list:
    file_path = os.path.join(PREDICT_DIR, test_file)
    input_image = load_img(file_path, 
                           target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_image = img_to_array(input_image)
    # the images are scaled during training so they need to be scaled for predictions too
    input_image = input_image/255.
    input_image = np.expand_dims(input_image, axis=0)
    prediction = json_model.predict(input_image)
    top_3 = prediction[0].argsort()[-3:][::-1]
    print('\nPredictions for {}:'.format(test_file))
    for idx in top_3:
        print(lookup_dict.get(idx))


Predictions for Audi-100_Sedan-1994.jpg:
Audi-100_Sedan-1994
Audi-100_Wagon-1994
Audi-V8_Sedan-1994

Predictions for Aston_Martin-Martin_V8_Vantage_Convertible-2012.jpg:
Aston_Martin-Martin_V8_Vantage_Convertible-2012
Aston_Martin-Martin_Virage_Convertible-2012
Aston_Martin-Martin_V8_Vantage_Coupe-2012

Predictions for Dodge-Ram_Pickup_3500_Crew_Cab-2010.jpg:
Dodge-Ram_Pickup_3500_Crew_Cab-2010
Dodge-Ram_Pickup_3500_Quad_Cab-2009
Ford-E_Series_Wagon_Van-2012

Predictions for AM-General_Hummer_SUV-2000.jpg:
AM-General_Hummer_SUV-2000
HUMMER-H2_SUT_Crew_Cab-2009
HUMMER-H3T_Crew_Cab-2010


## ROUND 3 of training (fine-tuned twice)

In [3]:
JSON_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT2_20_30.json')
with open (JSON_PATH, 'r') as json_file:
    loaded_model_json = json_file.read()

json_model = model_from_json(loaded_model_json)

In [4]:
WEIGHT_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT2_20_30.h5')
json_model.load_weights(WEIGHT_PATH)

### Predict!

In [5]:
test_file_list = [f for f in os.listdir(PREDICT_DIR) if\
                   os.path.isfile(os.path.join(PREDICT_DIR, f))]

with open(LOOKUP_PATH, 'rb') as f:
    lookup_dict = pickle.load(f)

for test_file in test_file_list:
    file_path = os.path.join(PREDICT_DIR, test_file)
    input_image = load_img(file_path, 
                           target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_image = img_to_array(input_image)
    # the images are scaled during training so they need to be scaled for predictions too
    input_image = input_image/255.
    input_image = np.expand_dims(input_image, axis=0)
    prediction = json_model.predict(input_image)
    top_3 = prediction[0].argsort()[-3:][::-1]
    print('\nPredictions for {}:'.format(test_file))
    for idx in top_3:
        print(lookup_dict.get(idx))


Predictions for Audi-100_Sedan-1994.jpg:
Audi-100_Sedan-1994
Mercedes_Benz-300_Class_Convertible-1993
Volvo-240_Sedan-1993

Predictions for Aston_Martin-Martin_V8_Vantage_Convertible-2012.jpg:
Aston_Martin-Martin_V8_Vantage_Convertible-2012
Aston_Martin-Martin_V8_Vantage_Coupe-2012
Aston_Martin-Martin_Virage_Convertible-2012

Predictions for Dodge-Ram_Pickup_3500_Crew_Cab-2010.jpg:
Dodge-Ram_Pickup_3500_Crew_Cab-2010
Dodge-Ram_Pickup_3500_Quad_Cab-2009
Dodge-Dakota_Club_Cab-2007

Predictions for AM-General_Hummer_SUV-2000.jpg:
AM-General_Hummer_SUV-2000
HUMMER-H2_SUT_Crew_Cab-2009
HUMMER-H3T_Crew_Cab-2010


## ROUND 4 of training (fine-tuned thrice)

In [3]:
JSON_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT3_30_40.json')
with open (JSON_PATH, 'r') as json_file:
    loaded_model_json = json_file.read()

json_model = model_from_json(loaded_model_json)

In [4]:
WEIGHT_PATH = os.path.join(SAVE_DIR, 'InceptionV3_21_FT3_30_40.h5')
json_model.load_weights(WEIGHT_PATH)

### Predict!

In [5]:
test_file_list = [f for f in os.listdir(PREDICT_DIR) if\
                   os.path.isfile(os.path.join(PREDICT_DIR, f))]

with open(LOOKUP_PATH, 'rb') as f:
    lookup_dict = pickle.load(f)

for test_file in test_file_list:
    file_path = os.path.join(PREDICT_DIR, test_file)
    input_image = load_img(file_path, 
                           target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_image = img_to_array(input_image)
    # the images are scaled during training so they need to be scaled for predictions too
    input_image = input_image/255.
    input_image = np.expand_dims(input_image, axis=0)
    prediction = json_model.predict(input_image)
    top_3 = prediction[0].argsort()[-3:][::-1]
    print('\nPredictions for {}:'.format(test_file))
    for idx in top_3:
        print(lookup_dict.get(idx))


Predictions for Audi-100_Sedan-1994.jpg:
Audi-100_Sedan-1994
Audi-100_Wagon-1994
Audi-V8_Sedan-1994

Predictions for Aston_Martin-Martin_V8_Vantage_Convertible-2012.jpg:
Aston_Martin-Martin_V8_Vantage_Convertible-2012
Aston_Martin-Martin_V8_Vantage_Coupe-2012
Audi-TTS_Coupe-2012

Predictions for Dodge-Ram_Pickup_3500_Crew_Cab-2010.jpg:
Dodge-Ram_Pickup_3500_Crew_Cab-2010
Dodge-Ram_Pickup_3500_Quad_Cab-2009
Dodge-Durango_SUV-2007

Predictions for AM-General_Hummer_SUV-2000.jpg:
AM-General_Hummer_SUV-2000
HUMMER-H2_SUT_Crew_Cab-2009
HUMMER-H3T_Crew_Cab-2010
